In [1]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# Укажите ваш email (обязательно для использования Entrez)
Entrez.email = "your.email@example.com"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

37879


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [6]:
# add_histone = (
#     "INSERT INTO histone "
#     "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
#     "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
# )
# add_histone_description = (
#     "INSERT INTO histone_description "
#     "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
#     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
# )
add_publication = (
    "INSERT INTO publication "
    "(id, title, doi, author, year) "
    "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
)
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
add_alternate_names = (
    "INSERT INTO alternative_name "
    "(name, taxonomy, gene, splice, histone) "
    "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
)
add_histone_has_publication = (
    "INSERT INTO histone_has_publication "
    "(histone_id, publication_id) "
    "VALUES (%s, %s)"
)

In [ ]:
def get_taxonomy_data(record):
    import re
    import sys

    taxonomy_data = {}
    taxonomy_data["organism"] = record.annotations["organism"]
    try:
        for a in record.features[0].qualifiers["db_xref"]:
            text = re.search("(\S+):(\S+)", a).group(1)
            taxid = re.search("(\S+):(\S+)", a).group(2)
            if text == "taxon":
                print(f"Fetched taxid from NCBI {taxid}")
                taxonomy_data["taxonomy_id"] = int(taxid)
            else:
                continue
    except:
        print("!!!!!!Unable to get TAXID for this record setting it to 1")
        taxonomy_data["taxonomy_id"] = 1  # unable to identify

    lineage = dict()
    for i in range(10):
        try:
            handle = Entrez.efetch(
                id=taxonomy_data["taxonomy_id"], db="taxonomy", retmode="xml"
            )
            tax_data = Entrez.read(handle)
            lineage = {
                d["Rank"]: d["ScientificName"]
                for d in tax_data[0]["LineageEx"]
                if d["Rank"] in ["class", "phylum"]
            }
            break
        except:
            print(
                "Unexpected error: {}, Retrying, attempt {}".format(
                    sys.exc_info()[0], i
                )
            )
            if i == 9:
                print(
                    f"FATAL ERROR could not get class and phylum from NCBI after 10 attempts for taxid:{taxonomy_data['taxonomy_id']}. Will add None for class and phylum!"
                )
            else:
                continue
    taxonomy_data["phylum"] = lineage.get("phylum", None)
    taxonomy_data["class"] = lineage.get("class", None)
    if taxonomy_data["phylum"] is not None:
        taxonomy_data["phylum"] = str(taxonomy_data["phylum"])
    if taxonomy_data["class"] is not None:
        taxonomy_data["class"] = str(taxonomy_data["class"])
    return taxonomy_data

In [8]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE name='cid'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,65,cid,None,None,None,cenH3_(Plants)


# Остановилась на этом моменте:

- **Название:** Unequal contribution of two paralogous CENH3 variants in cowpea centromere function
- **Год:** 2020
- **Автор:** Ishii

### **Примеры организмов с двумя CENH3-генами**  
1. **Растения**:  
   - <span style="color:green">*Arabidopsis lyrata* (двоюродный родственник *A. thaliana*) – ссылка **[9]**  </span>
   - *Luzula nivea* (ситник белоснежный) – **[???]**  
   - <span style="color:green">Ячмень (*Hordeum vulgare*) – **[11,15]**  
       - sanei_loss_2011 говорит о наличии двух генов у H. vulgare and H. bulbosum, диверсификация которых вероятно произошла около ~7 mya.
       - karimi-ashtiyani_point_2015 исследует значимость мутации L92F в H. vulgare (в ней есть полезная сстылка на статью ishii_differential_2015)</span>
   - <span style="color:green">Рожь (*Secale cereale*) – **[14]**  </span>
   - <span style="color:green">Горох (*Pisum sativum*) и *Lathyrus sativus* – **[12, 13]**  </span>

2. **Животные**:  
   - В разделе **Discussion** авторы отмечают, что множественные копии **CENH3/CENP-A** встречаются и у животных:  
     - <span style="color:green">*Caenorhabditis elegans* и *C. remanei* (нематоды) – **[28,30]**  </span>
     - <span style="color:green">Представители семейства **Bovidae** (коровьи) – **[29]**  </span>
     - *Drosophila* (мухи-дрозофилы) – **[31]**  

### **Ключевые выводы**  
- У этих организмов дупликация **CENH3** не всегда приводит к псевдогенизации одного из генов.  
- В некоторых случаях оба гена остаются функциональными, возможно, из-за **субфункционализации** (разделения функций) или **неофункционализации**.  
- Например, у *Drosophila* дупликации **CENH3 (Cid)** происходили независимо несколько раз, и разные варианты могут выполнять специализированные роли **[31]**.  

### **Ссылки на статьи**
Номера в квадратных скобках (например, [9], [31]) соответствуют списку литературы в оригинальной статье. Если вам нужны полные библиографические данные по конкретной ссылке, уточните номер — я могу выписать полное цитирование.  

*(Этот ответ объединяет информацию из нескольких разделов статьи, включая Introduction и Discussion.)*

# To Do
    
## <span style="color:black">Add publicationes for added histone nodes</span>

## <span style="color:black">Написать полное описание cenH3</span>

**deposition:** Про шапероны

**interactions:** Про кинетохору (есть статья с моделями сборки кинетохоры в разных видах живых организмов)

**caveats:** Разобраться с тем, как связаны cenH3 с транскрипцией центромерной ДНК [[Diverse mechanisms of centromere specification](https://www.notion.so/Diverse-mechanisms-of-centromere-specification-ff20544bd29e46138b5c696198d7857e?pvs=21) ].

1. Экспресия CENP-A происходит в раннюю G1 фазу. Вместе с тем, активно экспрессируется и шаперон HJURP, который встраивает CENP-A в нуклеосомы.
2. HJURP важен не просто для вставки CENP-A в нуклеосому, но и для определения локализации центромеры. Это происходит благодаря взаимодействию между HJURP и плотным комплексом, который состоит из белков Mis18α, Mis18β и M18BP1. Можно сказать, что комплекс Mis18 првлекает к себе HJURP, который в свою очередь уже связан с тетрамером CENP-A:H4.


# Close connections

In [35]:
cursor.close()
conn.close()
tunnel.stop()